In [ ]:
# downloadign kaggle and stuff

# !pip install kaggle
from google.colab import files

uploaded = files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



!kaggle datasets download -d msambare/fer2013
!unzip fer2013.zip -d fer2013

### Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import os
from tensorflow.keras.preprocessing import image


train_dir = "/content/fer2013/train"
test_dir = "/content/fer2013/test"

IMG_SIZE = 48
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    rescale=1./255,
    validation_split=0.2,
    fill_mode='nearest'
)
validation_datagen = ImageDataGenerator(rescale = 1./255,
                                         validation_split = 0.2)

##Testing the image augmentation here



import os
image_dirs = [os.path.join(train_dir, file) for file in os.listdir(train_dir)]

# Choose one image file as an example
example_dir = image_dirs[0]
image_files = [os.path.join(example_dir, file) for file in os.listdir(example_dir)]
example_file = image_files[0]
print(example_file)
img = image.load_img(example_file, target_size=(IMG_SIZE, IMG_SIZE))

img_array = image.img_to_array(img)
img_array = img_array.reshape((1,) + img_array.shape)

augmented_generator = train_datagen.flow(img_array, batch_size=1)

augmented_images = [next(augmented_generator)[0].astype(int) for _ in range(5)]

plt.figure(figsize=(10, 5))

# Original image
plt.subplot(1, 6, 1)
plt.imshow(img)
plt.title('Original')

# Augmented images
for i in range(5):
    plt.subplot(1, 6, i + 2)
    plt.imshow(augmented_images[i].astype('uint8'))
    plt.title(f'Augmented {i + 1}')

plt.show()

# setting up the flow from directories

train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size=(IMG_SIZE,IMG_SIZE),
                                                    batch_size=BATCH_SIZE,
                                                    color_mode="grayscale",
                                                    class_mode = "categorical",
                                                    subset = "training")
validation_generator = validation_datagen.flow_from_directory(directory = test_dir,
                                                    target_size=(IMG_SIZE,IMG_SIZE),
                                                    batch_size=BATCH_SIZE,
                                                    color_mode="grayscale",
                                                    class_mode = "categorical",
                                                    subset = "validation")


import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam

base_model = Sequential()

input_layer = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 1))

# Conv layer 1
x = Conv2D(32, kernel_size=(3, 3), activation='relu')(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)

# Conv layer 2
x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)

# Conv layer 3
x = Conv2D(128, kernel_size=(3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

# Conv layer 4
x = Conv2D(256, kernel_size=(3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)

# Conv layer 5
x = Conv2D(512, kernel_size=(3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

# Conv layer 6
x = Conv2D(512, kernel_size=(3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

x = Flatten()(x)

# Dense layer 1
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

# Dense layer 2
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

output_layer = Dense(7, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    filepath='model_weights.h5',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=True,
    mode='max',
    verbose=1
)

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[checkpoint_callback]
)

### Plotting some stuff using matplotlib code i found online

# Plot the train and validation loss
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot the train and validation accuracy
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, train_acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

import seaborn as sns
from sklearn.metrics import confusion_matrix

# Get the true labels and predicted labels for the validation set
validation_labels = validation_generator.classes
validation_pred_probs = model.predict(validation_generator)
validation_pred_labels = np.argmax(validation_pred_probs, axis=1)

# Compute the confusion matrix
confusion_mtx = confusion_matrix(validation_labels, validation_pred_labels)
class_names = list(train_generator.class_indices.keys())
sns.set()
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='YlGnBu',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

###Timing it

import time

img_path = '/content/fer2013/test/happy/PrivateTest_14332357.jpg'

img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE), color_mode='grayscale')

img_array = img_to_array(img)
img_array = img_array.reshape((1, IMG_SIZE, IMG_SIZE, 1))

img_array = img_array/255.0

start_time = time.time()
prediction = model.predict(img_array)
end_time = time.time()

model.save('model_weights.h5')

used_time = end_time - start_time

print("Used ", used_time, " seconds")

print("Predicted ", np.argmax(prediction))

